# **Introduction**

Texte introductif sur notre sujet, notre problématique, notre plan (angles d'attaque, études menées, bases de données utilisées)

# **I. Préparation de l'espace de travail**

## **1. Importations des modules et bibliothèques**

In [ ]:
# Installation des modules nécessaires à l'exécution du code

!pip install openpyxl cartiflette mapclassify

In [ ]:
# Import des bibliothèques utilisés pour la lecture des données et les statistiques descriptives 

import openpyxl 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from cartiflette import carti_download
import mapclassify
import plotly.express as px

## **2. Lecture et nettoyage des différentes bases de données**

### **1. Base de la démographie des médecins : demographie_medecins.xslx**

In [ ]:
# Import des bases de données depuis "Bases de données/"

df_medecins_effectif_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=1)
df_medecins_age_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=2)
df_medecins_densite_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=3)
df_medecins_secteur = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=6)

In [ ]:
# Création de plusieurs bases de données réduites (MG = médecins généralistes).

# 1. Base des effectifs des médecins généralistes.
df_MG_effectif = df_medecins_effectif_complet[
    (df_medecins_effectif_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_effectif_complet['specialites'] == '00-Ensemble')
    ]

df_MG_effectif.drop(axis = 1, columns = ['specialites', 'specialites_agregees'], inplace = True)
df_MG_effectif.reset_index(drop = True, inplace = True)

# 2. Base des âges moyens des médecins généralistes (ici, on ne considérera pas le sexe).
df_MG_age = df_medecins_age_complet[
    (df_medecins_age_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_age_complet['sexe'] == '0-Ensemble')
    & (df_medecins_age_complet['specialites'] == '00-Ensemble')
    ]

df_MG_age.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_MG_age.reset_index(drop = True, inplace = True)

# **II. Statistiques descriptives**

## **1. Étude de l'offre médicale globale en France**

### **a. Étude démographique de l'offre médicale globale en France**

#### **Évolution du nombre de médecins généralistes en France entre 2012 et 2025**

<p style="color:red">Est-ce qu'on crée vraiment une fonction par type de graphique ou flemme ?</p>

In [ ]:
# Définition d'une fonction pour le diagramme circulaire (ou "pie chart")
def plot_camembert(df, annee, ax, x, y):
    df_annuel = df[[x, y]]

    labels = df_annuel[x].unique()
    sizes = df_annuel[y].unique()

    ax.pie(sizes, labels=labels, autopct='%1.1f%%')
    ax.axis('equal')
    ax.set_title(f"{annee}")

In [ ]:
# Dans un premier temps, ou souhaite étudier l'évolution du nombre de médecins généralistes en France.
# On construit la base de données qui contient le nombre de médecins généralistes dans l'ensemble du territoire français par année.
df = df_MG_effectif.copy()

df = df[
    (df['exercice']=='0-Ensemble') 
    & (df['tranche_age']=='00-Ensemble') 
    & (df['region'] == '00-Ensemble')
    & (df['departement'] == '000-Ensemble') 
    & (df['sexe'] == '0-Ensemble')
    & (df['territoire'] == "0-France entière")
    ] 

df

In [ ]:
# 1. Définition des abscisses : années 2012 à 2025.
annees = list(range(2012,2026)) 

# 2. Définition des ordonnées : effectifs des années 2012 à 2025.
effectifs = df[[f"effectif_{a}" for a in annees]].values.flatten() 

# 3. Tracé du graphique.
plt.plot(annees, effectifs, marker = "o") 

# 4. Titre et axes.
plt.ylabel("Nombre de médecins généralistes") 
plt.title("Évolution du nombre de médecins généralistes en France") 

# 5. Légende.
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en France est passé d'environ 101 500 en 2012 à 100 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

#### **Évolution de l'âge moyen des généralistes en France entre 2012 et 2015**

In [ ]:
# On s'intéresse à l'évolution de l'âge moyen des MG dans l'ensemble de la France, tous exercices confondus.
df = df_MG_age.copy()

df = df[
    (df['exercice']=='0-Ensemble') 
    & (df['region'] == '00-Ensemble') 
    & (df['territoire'] == "0-France entière")
]

df

In [ ]:
# 1. Définition des abscisses : années 2012 à 2025.
annees = list(range(2012,2026)) 

# 2. Définition des ordonnées : effectifs des années 2012 à 2025.
effectifs = df[[f"am_{a}" for a in annees]].values.flatten() 
plt.plot(annees, effectifs, marker = "o") 

# 3. Titre et axes. 
plt.ylabel("Âge moyen des médecins généralistes") 
plt.title("Évolution de l'âge moyen des médecins généralistes en France") 

# 4. Légende.
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : L'âge moyen des médecins généralistes en France est passé de 51,1 ans en 2012 à 50,4 ans en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

#### **Évolution des tranches d'âge des médecin généralistes entre 2012 et 2025**

In [ ]:
# On s'intéresse maintenant à l'évolution des tranches d'âge des médecins généralistes.
# On construit la base de données qui contient le nombre de médecins généralistes dans l'ensemble du territoire français, par année et par tranche d'âge.
df = df_MG_effectif.copy()

df = df[
    (df['exercice']=='0-Ensemble') 
    & (df['region'] == '00-Ensemble') 
    & (df['departement'] == '000-Ensemble') 
    & (df['territoire'] == "0-France entière")
    & (df['tranche_age'] != "00-Ensemble")
    & (df['sexe'] == '0-Ensemble')
    ]

# On identifie de nouveaux groupes d'âge.
# 1. On crée une fonction pour attribuer les nouveaux groupes correspondants aux tranches d'âge.
def nouvelle_tranche(tranche):
    if tranche in ["01-moins de 30 ans", "02-entre 30 et 34 ans"]:
        return "1. Moins de 35 ans"
    elif tranche in ["03-entre 35 et 39 ans", "04-entre 40 et 44 ans"]:
        return "2. 35–44 ans"
    elif tranche in ["05-entre 45 et 49 ans", "06-entre 50 et 54 ans"]:
        return "3. 45–54 ans"
    elif tranche in ["07-entre 55 et 59 ans", "08-entre 60 et 64 ans"]:
        return "4. 55–64 ans"
    else:
        return "5. 65 ans et +"

# 2. On somme les effectifs des tranches d'âge au sein du même groupe.
df["nouvelle_tranche_age"] = df["tranche_age"].apply(nouvelle_tranche)
cols_effectifs = [c for c in df.columns if c.startswith("effectif_")]
df = df.groupby("nouvelle_tranche_age")[cols_effectifs].sum().reset_index()

# On renomme les colonnes pour plus de lisibilité.
df.columns = ["Tranche d'âge"] + [f"{a}" for a in range(2012, 2026)]

df

<p style="color:red">Remettre la position de la légende (pas très lisible)</p>

In [ ]:
# 1. Définition des abscisses :  années une sur deux entre 2013 et 2025.
annees = list(range(2013, 2026, 2))

cols = [f"{a}" for a in annees]

# 2. On prend la transposée de la base de données pour faire l'histogramme.
df_plot = df.set_index("Tranche d'âge")[cols].T

# 3. Tracé de l'histogramme en barres empilées.
df_plot.plot(kind="bar",stacked=True,figsize=(10, 6))

# 4. Titre et axes.
plt.ylabel("Proportion")
plt.xlabel("Année")
plt.title("Répartition des médecins généralistes par tranche d’âge\n(une année sur deux)")
plt.legend(title="Tranche d’âge", bbox_to_anchor=(1.05, 1))

# 5. Légende.
plt.text(0, -0.28, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.33, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.55, "Lecture : Entre 2013 et 2025, le corps des médecins généralistes en exercice se transforme\n"
    "avec une baisse de la proportion des 45–64 ans et une hausse des moins de 35 ans et des 65 ans et plus. Cette \n"
    "évolution montre à la fois l'arrivée d'une nouvelle génération et un vieillissement partiel \n"
    "de la profession.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.tight_layout()
plt.show()


#### **Évolution de la parité chez les médecins généralistes en France.**

<p style="color:red">Je pense qu'on peut directement utiliser les données du tableau sans graphe pour expliquer le camembert : on passe de 41k à 52,5k de femmes MG et de 60k à 47k d'hommes MG. Donc augmentation de femmes MG et baisse d'hommes MG, à voir pourquoi.</p>

In [ ]:
# On étudie l'évolution de la parité au sein des médecins généralistes.
# On construit la base de données qui contient le nombre de médecins généralistes dans l'ensemble du territoire français, par année et par sexe.
df = df_MG_effectif.copy()

df = df[
    (df['sexe'] != '0-Ensemble')
    & (df['departement'] == '000-Ensemble')
    & (df['exercice']=='0-Ensemble') 
    & (df['region'] == '00-Ensemble') 
    & (df['territoire'] == "0-France entière")
    & (df['tranche_age'] == "00-Ensemble")
    ]

df.reset_index(drop = True, inplace = True)

df

In [ ]:
# 1. Définition de la figure (répartition et taille)
fig, axes = plt.subplots(1, 2, figsize=(15, 10))

# 2. Tracé des graphes
for axe, annee in zip(axes, [2012, 2025]):
    plot_camembert(df, annee, axe, 'sexe', f'effectif_{annee}')

# 3. Titre
fig.suptitle("Parité homme/femme chez les médecins généralistes", fontsize=20, )

# 4. Légende
plt.text(0, -0.19, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.23, "Champ : Médecins en généraliste France, DROM exclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.32,
    "Lecture : Entre 2012 et 2025, la proportion de femmes chez les médecins généralistes\n"
    "est passée de 40.8% à 52.4%. \n",
    va="bottom",
    fontsize=9,
    transform=plt.gca().transAxes
)

plt.tight_layout()
plt.show()

### **b. Étude géographique de l'offre médicale globale en France**

#### **Évolution du nombre de médecins généralistes par région**

In [ ]:
# On souhaite étudier l'évolution du nombre de médecis généralistes par région.
# On construit une base de données qui contient les effectifs par région par année.
df = df_MG_effectif.copy()

df = df[
    (df['exercice']=='0-Ensemble')
    & (df['tranche_age']=='00-Ensemble') 
    & (df['sexe']=='0-Ensemble')
    & (df['departement']=='000-Ensemble')
    & (df['region'] != '00-Ensemble')
    ]

df.sort_values('region', inplace=True)
df.reset_index(inplace=True, drop=True)

df

In [ ]:
# 1. Définition des abscisses : années 2012 à 2025.
annees = list(range(2012,2026))

# 2. Définition dune palette avec 20 couleurs de manière à ce que chaque région ait une couleur différente.
plt.gca().set_prop_cycle(color=plt.cm.tab20.colors)

# 3. Définition des ordonnées : effectifs par région des années 2012 à 2025.
for region in df['region'].unique() :
    df_region = df[(df['region'] == region) & (df['departement'] == '000-Ensemble')]
    effectifs = df_region[[f"effectif_{a}" for a in annees]].values.flatten()
    plt.plot(annees, effectifs, label = region, marker = "o")

# 4. Titre et axes. 
plt.xlabel("Année")
plt.ylabel("Nombre de médecins généralistes")
plt.title(f"Évolution du nombre de médecins généralistes par région")

# Légende.
plt.legend(title="Région", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes par région en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en Île de France est passé d'environ 18 500 en 2012 à 16 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

##### **1.3) Etude des spécialisations & sectorisation de l'offre médicale globale en France**

**1.3.1) Etude de la sectorisation des médecins en France**

In [ ]:
df_medecins_secteur_réduit= df_medecins_secteur[(df_medecins_secteur['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_secteur['region'] == '00-Ensemble')
    & (df_medecins_secteur['departement'] == '000-Ensemble')
    & (df_medecins_secteur['mode_exercice'] == '0-Ensemble')
    & (df_medecins_secteur['territoire'] == '0-France entière')
    & (df_medecins_secteur['specialites'] == '01-Médecine générale')
    ]
df_medecins_secteur_réduit.head()

In [ ]:
df_medecins_secteur_réduit = df_medecins_secteur_réduit[df_medecins_secteur_réduit["secteur_activite"] != "00-Ensemble"]

# Définition de la figure (répartition & taille)
fig, axes = plt.subplots(2, 2, figsize=(17, 10))

# Tracé des graphes
for axe, annee in zip(axes.flatten(), [2012, 2017, 2021, 2025]):
    plot_camembert(df_medecins_secteur_réduit, annee, axe, 'secteur_activite',f'activités_{annee}')

# Titre
fig.suptitle("Répartition des médecins par secteur d’activité", fontsize=20, fontweight="bold")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généraliste en France, DROM exclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.38,
    "Lecture : Entre 2012 et 2025, la médecine générale évolue nettement au profit\n"
    "des formes collectives, au détriment du cabinet individuel. La montée des\n"
    "cabinets de groupe et des remplaçants traduit un changement structurel des\n"
    "modes d’exercice, l’hôpital public demeurant globalement stable.",
    va="bottom",
    fontsize=9,
    transform=plt.gca().transAxes
)

plt.tight_layout()
plt.show()

In [ ]:
df_hopital = df_medecins_secteur[(df_medecins_secteur["secteur_activite"] == "01-Hôpital public")
    & (df_medecins_secteur["mode_exercice"] == "0-Ensemble")
    & (df_medecins_secteur["departement"] == "000-Ensemble")
    & (df_medecins_secteur["region"] == "00-Ensemble")
    & (df_medecins_secteur["territoire"] == "0-France entière")
    & (df_medecins_secteur["specialites_agregees"] == "00-Ensemble")
    & (df_medecins_secteur["specialites"] == "00-Ensemble")
    ]

# Colonnes d'activité
cols_activite = [c for c in df_hopital.columns if c.startswith("activités_")]

# Années
annees = [int(c.split("_")[1]) for c in cols_activite]


plt.figure(figsize=(8, 5))
plt.plot(annees, df_hopital[cols_activite].iloc[0], marker='o')


plt.xlabel("Année")
plt.ylabel("Activité hospitalière")
plt.title("Évolution de l’activité hospitalière")
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins en France, toute spécialité confondues, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : L'activité hospitalière suit une croissance quasi-linéaire depuis 2012 en France.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


**1.3.1) Etude de la spécialisation des médecins en France**

In [ ]:
df_medecins_specialité = df_medecins_effectif_complet[(df_medecins_effectif_complet['sexe'] == '0-Ensemble')
    & (df_medecins_effectif_complet['departement'] == '000-Ensemble')
    & (df_medecins_effectif_complet['exercice']=='0-Ensemble') 
    & (df_medecins_effectif_complet['region'] == '00-Ensemble') 
    & (df_medecins_effectif_complet['territoire'] == "0-France entière")
    & (df_medecins_effectif_complet['tranche_age'] == "00-Ensemble")
    & (df_medecins_effectif_complet['specialites'] != '00-Ensemble')
    ]

df_medecins_specialité.drop(axis = 1, columns = ['region','specialites_agregees','sexe', 'tranche_age','territoire','departement','exercice'], inplace = True)
df_medecins_specialité.reset_index(drop = True, inplace = True)
df_medecins_specialité.head()

In [ ]:
# Définition de la figure (répartition & taille)
fig, axes = plt.subplots(3, 1, figsize=(20, 20))

# Tracé des graphes
for axe, annee in zip(axes, [2012, 2019, 2025]):
    plot_camembert(df_medecins_specialité, annee, axe, 'specialites', f'effectif_{annee}')

# Titre
fig.suptitle("Répartition des médecins par spécialité", fontsize=20, fontweight="bold")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins en France, DROM exclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.32,
    "Lecture : Entre 2012 et 2025, la médecine générale demeure la spécialité \n"
    "majoritaire mais voit sa part reculer progressivement. Cette baisse s’accompagne \n"
    "d’une diversification des autres spécialités.\n",
    va="bottom",
    fontsize=9,
    transform=plt.gca().transAxes
)

plt.tight_layout()
plt.show()

#### **2) Etude de la présence de déserts médicaux**